# Plot cross-section profile of slip

2025.3.24 Kurama Okubo

In [ ]:
import os
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
import matplotlib.colors as colors
import matplotlib as mpl

%matplotlib inline
import numpy as np
import pandas as pd
import datetime
from datetime import timedelta
from tqdm import tqdm
import warnings
import time

from importParaviewColormap import importParaviewColormap
import seaborn as sns
from matplotlib.collections import LineCollection

%load_ext autoreload
%autoreload 2


plt.rcParams["font.family"] = 'Arial'
# plt.rcParams["font.sans-serif"] = "DejaVu Sans, Arial, Helvetica, Lucida Grande, Verdana, Geneva, Lucid, Avant Garde, sans-serif"
plt.rcParams["font.size"] = 10
plt.rcParams["xtick.direction"] = "in"
plt.rcParams["xtick.major.size"] = 4.75
plt.rcParams["xtick.major.width"] = 0.75
plt.rcParams["xtick.minor.size"] = 3
plt.rcParams["xtick.minor.width"] = 0.4
plt.rcParams["xtick.minor.visible"] = True

plt.rcParams["ytick.direction"] = "in"
plt.rcParams["ytick.major.size"] = 4.75
plt.rcParams["ytick.major.width"] = 0.75
plt.rcParams["ytick.minor.size"] = 3
plt.rcParams["ytick.minor.width"] = 0.4
plt.rcParams["ytick.minor.visible"] = True

plt.rcParams["savefig.transparent"] = True

plt.rcParams['axes.linewidth'] = 0.75

In [ ]:
figdir = "../figure/aux03_cross-section"
if not os.path.exists(figdir):
    os.makedirs(figdir)

In [ ]:
E = 96e9
nu = 0.246 # metagabbro
mu = E/(2*(1+nu))

a_patch = 4.0e-3
a_nuc = 2.5e-3 # nucleation radius 

nb_x_elements = 1024 #128 #1024
nb_z_elements = 1024 #128 #1024


Ifmasteroutput=True # add file key to the master output
if Ifmasteroutput:
    filekey = "_master"
else:
    filekey = ""

In [ ]:
# case study parameter casename

a_patch = 4.0e-3
a_nuc = 2.5e-3
a_margin = 4.08e-3

rupturetype = "pulse"
pdcscaling = 0.60 #0.65
bgbeta= 0.35 #0.4

sig_n = 6e6

IfBinaryOutput = True

casestr_pulse = f"a={a_patch*1e3:.2f}_ruptype={"pulse"}_pdcscaling={pdcscaling:.3f}_sn={sig_n/1e6:.1f}MPa_hatfr=0.3_bgbeta={bgbeta:.2f}"
casestr_crack = f"a={a_patch*1e3:.2f}_ruptype={"crack"}_pdcscaling={pdcscaling:.3f}_sn={sig_n/1e6:.1f}MPa_hatfr=0.3_bgbeta={bgbeta:.2f}"

finame=f"../../preprocess_modelsetup/data/gouge_dynamicrupture_modelparam_{casestr_pulse}{filekey}.csv"

# Read model parameters
df_modelparam = pd.read_csv(finame, index_col=0)


datadir_root = "/Volumes/Okuboetal2025_masterHDD/RuptureSimulation/main_casestudy/build_hpcv62_mastercase_v3/simulations_main_casestudy_hpc_master"


In [ ]:
df_modelparam.head()

In [ ]:
# Select gouge event id to plot the snapshots
expr_id = 87
gougeevent_id = 129 # 24, 50, 52, 72, 129

In [ ]:
df_modelparam_selected = df_modelparam[df_modelparam.index == gougeevent_id]

# if ifParamStudy:
simulation_name_pulse = f"fb03-{expr_id:03d}__{df_modelparam_selected.index[0]:04d}_{casestr_pulse}_{df_modelparam_selected.delsigma_factor.values[0]:.4f}"
simulation_name_crack = f"fb03-{expr_id:03d}__{df_modelparam_selected.index[0]:04d}_{casestr_crack}_{df_modelparam_selected.delsigma_factor.values[0]:.4f}"

print(f"Process {gougeevent_id}: {simulation_name_pulse} and {simulation_name_crack}")


# Read profile

We used `offset` and `count` to partially read binary file to increase the i/o speed.

In [ ]:
# time and coordinates are same for pulse and crack cases

df_time = pd.read_csv(os.path.join(datadir_root,simulation_name_pulse+".time"), header=None, sep=' ', index_col=0)
df_coord = pd.read_csv(os.path.join(datadir_root,simulation_name_pulse+".coords"), header=None, sep=' ', index_col=None)
NT=len(df_time)


In [ ]:
# location of the center of nucleation zone
nuc_x = -(a_patch - a_nuc)
nuc_z = 0

xcoord = df_coord.loc[:,0].values
zcoord = df_coord.loc[:,2].values

x_length = xcoord.max()
z_length = zcoord.max()

# ref also: https://stackoverflow.com/a/35176314
dgrid = (zcoord[1] - zcoord[0])
print(f"Grid size: {dgrid*1e3}[mm]") 

X = xcoord.reshape(nb_x_elements,nb_z_elements).T- dgrid/2 - x_length/2
Z = zcoord.reshape(nb_x_elements,nb_z_elements).T- dgrid/2 - z_length/2



In [ ]:
read_comp = 0 #0: Mode II direction, 1: normal (Mode I), 2:Mode III direction 

data_disp_all = dict()
tplot_list = [0.6e-6, 1.0e-6, 1.6e-6, 6.0e-6] # plot time list

for tplot in tplot_list:
    
    tind = np.where(df_time.values >= 0.99*tplot)[0][0] # search the time index of tplot
    
    Nchank = nb_x_elements * nb_z_elements
    Noffset = Nchank * tind * np.dtype(np.float32).itemsize # skip untile the plot timing
    
    # read displacement
    read_parameter = f"top_disp_{read_comp}" # select the parameter to read
    if IfBinaryOutput:
        disp_pulse = np.fromfile(os.path.join(datadir_root,simulation_name_pulse+f"-DataFiles/{read_parameter}.out"), dtype="float32", offset=Noffset, count=Nchank)
        disp_crack = np.fromfile(os.path.join(datadir_root,simulation_name_crack+f"-DataFiles/{read_parameter}.out"), dtype="float32", offset=Noffset, count=Nchank)
        # df_data_disp = pd.DataFrame(data=D.reshape((NT, -1)))
    else:
        warnings.warn("only binary output available in this notebook.")
        # df_data_disp = pd.read_csv(os.path.join(datadir_root,simulation_name_pulse+f"-DataFiles/{read_parameter}.out"), header=None, sep=' ', engine="c")

    data_disp_all[f"{tplot*1e6}us_pulse"] = disp_pulse 
    data_disp_all[f"{tplot*1e6}us_crack"] = disp_crack 

# Plot cross-section

In [ ]:
z_cross=0.0
trac_z_ind = np.where(Z[:, 0] >= z_cross)[0][0]

In [ ]:
value_factor_double = 2.0 # double the value for slip and slip velocity


fig, ax = plt.subplots(1, 1, figsize=(4.5, 3.4))

for i, tplot in enumerate(tplot_list):

    disp_crosssec_crack = data_disp_all[f"{tplot*1e6}us_crack"].reshape(nb_x_elements,nb_z_elements).T[trac_z_ind, :] 
    disp_crosssec_pulse = data_disp_all[f"{tplot*1e6}us_pulse"].reshape(nb_x_elements,nb_z_elements).T[trac_z_ind, :] 

    if i==0:
        label_c = "without self-healing"
        label_p = "with self-healing"
    else:
        label_c = ""
        label_p = ""
        
    ax.plot(X[0]*1e3, value_factor_double*disp_crosssec_pulse*1e6, c="crimson", lw=1.5, label=label_p)
    ax.plot(X[0]*1e3, value_factor_double*disp_crosssec_crack*1e6, c="k", lw=1, ls="--", label=label_c)
    
    ax.set_xlabel("x [mm]")
    ax.set_ylabel(r"Slip [$\mu$m]")
    ax.legend(loc=1)
    
    ax.set_xlim([-15, 15])
    ax.set_ylim([-0.02, 0.8])

ax.text(0.04, 0.925, f"GP event M{gougeevent_id}", transform=ax.transAxes)

plt.tight_layout()
plt.savefig(figdir + f"/cross-section_disp_{simulation_name_pulse}.png", dpi=80, bbox_inches="tight")
plt.savefig(figdir + f"/cross-section_disp_{simulation_name_pulse}.eps", bbox_inches="tight")

# Plot shear stress

In [ ]:
read_comp = 0 #0: Mode II direction, 1: normal (Mode I), 2:Mode III direction 

data_coh_all = dict()
tplot_list = [0.3e-6, 6.0e-6] # plot time list

for tplot in tplot_list:
    
    tind = np.where(df_time.values >= 0.99*tplot)[0][0] # search the time index of tplot
    
    Nchank = nb_x_elements * nb_z_elements
    Noffset = Nchank * tind * np.dtype(np.float32).itemsize # skip untile the plot timing
    
    # read displacement
    read_parameter = f"cohesion_{read_comp}" # select the parameter to read
    if IfBinaryOutput:
        coh_pulse = np.fromfile(os.path.join(datadir_root,simulation_name_pulse+f"-DataFiles/{read_parameter}.out"), dtype="float32", offset=Noffset, count=Nchank)
        coh_crack = np.fromfile(os.path.join(datadir_root,simulation_name_crack+f"-DataFiles/{read_parameter}.out"), dtype="float32", offset=Noffset, count=Nchank)
        # df_data_disp = pd.DataFrame(data=D.reshape((NT, -1)))
    else:
        warnings.warn("only binary output available in this notebook.")
        # df_data_disp = pd.read_csv(os.path.join(datadir_root,simulation_name_pulse+f"-DataFiles/{read_parameter}.out"), header=None, sep=' ', engine="c")

    data_coh_all[f"{tplot*1e6}us_pulse"] = coh_pulse 
    data_coh_all[f"{tplot*1e6}us_crack"] = coh_crack 

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4.5, 3.4))

for i, tplot in enumerate(tplot_list):

    coh_crosssec_crack = data_coh_all[f"{tplot*1e6}us_crack"].reshape(nb_x_elements,nb_z_elements).T[trac_z_ind, :] 
    coh_crosssec_pulse = data_coh_all[f"{tplot*1e6}us_pulse"].reshape(nb_x_elements,nb_z_elements).T[trac_z_ind, :] 

    if i==0:
        label_c = "without self-healing"
        label_p = "with self-healing"
    else:
        label_c = ""
        label_p = ""
        
    ax.plot(X[0]*1e3, coh_crosssec_pulse/1e6, c="crimson", lw=1.5, label=label_p)
    ax.plot(X[0]*1e3, coh_crosssec_crack/1e6, c="k", lw=1, ls="--", label=label_c)
    
    ax.set_xlabel("x [mm]")
    ax.set_ylabel(r"Shear stress [MPa]")
    ax.legend(loc=1)
    
    ax.set_xlim([-15, 15])
    ax.set_ylim([-0.02, 7])

ax.text(0.04, 0.925, f"GP event M{gougeevent_id}", transform=ax.transAxes)

plt.tight_layout()
plt.savefig(figdir + f"/cross-section_shearstress_{simulation_name_pulse}.png", dpi=80, bbox_inches="tight")
plt.savefig(figdir + f"/cross-section_shearstress_{simulation_name_pulse}.eps", bbox_inches="tight")